In [9]:
from pandas import read_csv
#Load data set
look_back = 2
series_influ_A_df = read_csv('../temp_data/influA_vietnam_last_10_days.csv', index_col=0, engine='python')
train_df = series_influ_A_df.iloc[0:735]
test_df = series_influ_A_df.iloc[735-look_back:]
# series_influ_A_df = series_influ_A_df["Influenza A - All types of surveillance"]

In [10]:
train_df.corr()

,Influenza A - All types of surveillance,temp,humidity,dew,precipcover,tempmax,tempmin,windspeed
Influenza A - All types of surveillance,1.000000,0.233416,0.016145,0.216907,0.019588,0.227861,0.231337,0.130241
temp,0.233416,1.000000,0.156404,0.950239,0.392861,0.992806,0.990106,0.269600
humidity,0.016145,0.156404,1.000000,0.454868,0.418620,0.088874,0.249285,0.122371
dew,0.216907,0.950239,0.454868,1.000000,0.487902,0.921801,0.971545,0.286277
precipcover,0.019588,0.392861,0.418620,0.487902,1.000000,0.359425,0.434643,0.141203
tempmax,0.227861,0.992806,0.088874,0.921801,0.359425,1.000000,0.968351,0.261132
tempmin,0.231337,0.990106,0.249285,0.971545,0.434643,0.968351,1.000000,0.281380
windspeed,0.130241,0.269600,0.122371,0.286277,0.141203,0.261132,0.281380,1.000000


In [11]:
series_influ_A_df.isna().sum()

Influenza A - All types of surveillance    1
temp                                       0
humidity                                   0
dew                                        0
precipcover                                0
tempmax                                    0
tempmin                                    0
windspeed                                  0
dtype: int64

In [12]:
series_influ_A_df = series_influ_A_df[["Influenza A - All types of surveillance", "temp", "tempmax", "dew","windspeed"]]

In [13]:
series_influ_A_df = series_influ_A_df.dropna()

series_influ_A_df = series_influ_A_df.astype('float32')
series_influ_A_df = series_influ_A_df.values
# normalize the dataset
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(series_influ_A_df)
# Create Training and Test

train = dataset[:735, :]
test = dataset[735-look_back:, :]

import numpy as np
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

In [14]:
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf

from keras.losses import mean_squared_error

def root_mean_squared_error(y_true, y_pred):
    return tf.sqrt(mean_squared_error(y_true, y_pred))
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(5, input_shape=(2, 5)))
model.add(Dense(5))
model.compile(loss=root_mean_squared_error, optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100


d:\my_study\gr3\DATN\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


733/733 - 2s - 3ms/step - loss: 0.2228
Epoch 2/100
733/733 - 1s - 1ms/step - loss: 0.1225
Epoch 3/100
733/733 - 1s - 1ms/step - loss: 0.1101
Epoch 4/100
733/733 - 1s - 1ms/step - loss: 0.1044
Epoch 5/100
733/733 - 1s - 1ms/step - loss: 0.1008
Epoch 6/100
733/733 - 1s - 1ms/step - loss: 0.0977
Epoch 7/100
733/733 - 1s - 1ms/step - loss: 0.0963
Epoch 8/100
733/733 - 1s - 1ms/step - loss: 0.0949
Epoch 9/100
733/733 - 1s - 2ms/step - loss: 0.0941
Epoch 10/100
733/733 - 1s - 2ms/step - loss: 0.0940
Epoch 11/100
733/733 - 1s - 1ms/step - loss: 0.0934
Epoch 12/100
733/733 - 1s - 1ms/step - loss: 0.0927
Epoch 13/100
733/733 - 1s - 1ms/step - loss: 0.0924
Epoch 14/100
733/733 - 1s - 1ms/step - loss: 0.0919
Epoch 15/100
733/733 - 1s - 1ms/step - loss: 0.0915
Epoch 16/100
733/733 - 1s - 1ms/step - loss: 0.0913
Epoch 17/100
733/733 - 1s - 1ms/step - loss: 0.0907
Epoch 18/100
733/733 - 1s - 1ms/step - loss: 0.0903
Epoch 19/100
733/733 - 1s - 1ms/step - loss: 0.0898
Epoch 20/100
733/733 - 1s - 1ms/s

In [16]:
model.save("../model/LSTM_4_4.keras")